In [1]:
import pandas as pd
import json

tweet_texts = []
with open('English_tweets.json', 'r') as jsonfile:
        tweets = json.load(jsonfile)
for tweet in tweets:
    tweet_texts.append(tweet['tweet_text'])

In [2]:
tweet_texts[0:10]

['RT @AC360: You said "what punishment of gods are not gifts. Do you really believe that?" @andersoncooper, choking back tears, asks Stephen…',
 '@carlarhoffmann @AC360 thank you!',
 'one of my favorite excerpts from my interview with @StephenAtHome. This question he embraces left me stunned when i… https://t.co/YV2s8z34Jk',
 'Come see me in #Winnipeg on Sunday, Sept 15 for a fun evening of conversation, sharing personal stories, as well as… https://t.co/fW9m3RKzKa',
 'RT @AC360: Megan Rapinoe says kneeling during the National Anthem was difficult, but not disrespectful, adding that she is hopeful there wi…',
 'excited to interview #MeganRapinoe live tonight on @AC360 8pm. what questions would you ask her? https://t.co/dq1N2ypzu1',
 'Thank you for all the lovely messages about my mom. I was lucky to have her for so long. She was the coolest lady i… https://t.co/hluZ1EJlgO',
 'Thanks for the cover @EW! https://t.co/bk5dYEggMx',
 'A great new book called #WeAreEverywhere  just came out. 

In [3]:
df = pd.DataFrame(tweet_texts)

In [4]:
df.columns = ['tweet_text']

In [5]:
import string, re
import textblob as tb

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    res = []
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = tb.TextBlob(text)
    
    for token, tag in text.pos_tags:
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        token = token.lemmatize(pos)
        res.append(token)

    return ' '.join(res)

df_clean = pd.DataFrame(df.tweet_text.apply(lambda x: clean_text(x)))

In [6]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word',       
                             min_df=3,                        # minimum required occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=5000,             # max number of unique words. Build a vocabulary that only consider the top max_features ordered by term frequency across the corpus
                            )

data_vectorized = vectorizer.fit_transform(df_clean['tweet_text'])

lda_model = LatentDirichletAllocation(n_components=10, # Number of topics
                                      learning_method='online',
                                      random_state=0,       
                                      n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

In [7]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')

C:\Users\sneha\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2      -1.505655  69.313271       1        1  13.602906
4      49.578903  30.024618       2        1  13.052644
0      42.374710  -4.986403       3        1  12.220954
1      16.138281  15.181349       4        1   9.862485
3     -33.498383  53.368210       5        1   9.191320
9     -40.707043  18.353722       6        1   9.178956
7      10.385882 -20.934681       7        1   8.934822
8      -7.273433  33.186008       8        1   8.794423
5     -15.814617  -2.142410       9        1   7.679706
6      24.677895  50.503052      10        1   7.481784, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
190   Default  5154.000000            amp  5154.000000  30.0000  30.0000
3508  Default  5626.000000      president  5626.000000  29.0000  29.0000
2693  Default  3716.000000           make  3716.000000  28.0000  28.0000
2221  Default  2477.000000          india  2477.000000  27.0000  27.0000
4974  Default  2647.000000           year  2647.000000  26.0000  26.0000
3329  Default  4293.000000         people  4293.000000  25.0000  25.0000
1924  Default  3120.000000          great  3120.000000  24.0000  24.0000
4931  Default  2038.000000          woman  2038.000000  23.0000  23.0000
989   Default  2130.000000        country  2130.000000  22.0000  22.0000
4008  Default  2528.000000            say  2528.000000  21.0000  21.0000
3113  Default  2492.000000          obama  2492.000000  20.0000  20.0000
4555  Default  2453.000000           time  2453.000000  19.0000  19.0000
4943  Default  2964.000000           work  2964.000000  18.0000  18.0000
4865  Default  2264.000000          watch  2264.000000  17.0000  17.0000
813   Default  1963.000000            com  1963.000000  16.0000  16.0000
3905  Default  2015.000000          right  2015.000000  15.0000  15.0000
3650  Default  2283.000000            que  2283.000000  14.0000  14.0000
1093  Default  2873.000000            day  2873.000000  13.0000  13.0000
4834  Default  1932.000000           vote  1932.000000  12.0000  12.0000
180   Default  2259.000000       american  2259.000000  11.0000  11.0000
4658  Default  3210.000000          trump  3210.000000  10.0000  10.0000
4571  Default  3674.000000          today  3674.000000   9.0000   9.0000
4500  Default  2050.000000          thank  2050.000000   8.0000   8.0000
688   Default  1587.000000         change  1587.000000   7.0000   7.0000
1890  Default  2380.000000           good  2380.000000   6.0000   6.0000
3247  Default  1871.000000           para  1871.000000   5.0000   5.0000
2614  Default  1471.000000           live  1471.000000   4.0000   4.0000
4388  Default  1511.000000        support  1511.000000   3.0000   3.0000
2985  Default  1504.000000   narendramodi  1504.000000   2.0000   2.0000
3023  Default  1767.000000           need  1767.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
1158  Topic10   856.048682       democrat   856.966642   2.5916  -3.9751
1102  Topic10   838.064581         debate   838.982509   2.5916  -3.9963
2015  Topic10   665.414015           hear   666.331967   2.5913  -4.2270
517   Topic10   590.344105           book   591.262022   2.5911  -4.3467
2069  Topic10   577.846743           hold   578.764688   2.5911  -4.3681
3105  Topic10   438.999757         number   439.917735   2.5906  -4.6429
684   Topic10   394.944633         chance   395.862581   2.5904  -4.7487
3720  Topic10   390.533597          ready   391.451530   2.5904  -4.7599
3473  Topic10   386.103885        pradesh   387.021829   2.5903  -4.7713
1873  Topic10   377.629286           glad   378.547213   2.5903  -4.7935
4960  Topic10   369.055530          write   369.973455   2.5902  -4.8165
1921  Topic10   357.493075       grateful   358.411003   2.5901  -4.8483


In [8]:
import numpy as np

# Show top 20 keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,make,work,watch,good,look,talk,know,join,like,health,care,day,address,forward,just,plan,congratulation,new,start,biden
Topic 1,right,day,people,let,stand,act,just,read,end,long,love,power,ask,continue,want,stop,gun,congress,violence,attack
Topic 2,president,people,say,obama,time,trump,american,family,today,way,state,think,leader,speak,happy,tell,america,community,thing,want
Topic 3,year,com,vote,win,election,realdonaldtrump,kamalaharris,brasil,trump,student,public,leave,south,city,remember,govt,poll,voter,china,republican
Topic 4,que,thank,para,need,presidente,fight,temer,michel,mais,ncia,aovivo,governo,uma,dont,young,donald,hoje,life,matter,estamos
Topic 5,change,live,news,life,don,climate,believe,share,report,bring,hope,justice,lose,late,potus,school,medium,work,est,rio
Topic 6,india,woman,job,joebiden,nation,tulsigabbard,house,democrat,debate,hear,book,hold,pass,number,chance,like,ready,pradesh,glad,write
Topic 7,country,narendramodi,shashitharoor,bjp,arvindkejriwal,best,meet,congress,proud,indian,government,today,delhi,visit,incindia,united,modi,add,break,speech
Topic 8,amp,support,big,party,economy,great,meeting,run,important,week,yesterday,history,lead,meetthepress,deal,miss,good,war,candidate,case
Topic 9,great,help,today,campaign,pay,thanks,wish,friend,million,child,issue,team,strong,high,future,joe,tax,record,serve,know


In [9]:
Topics_theme = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
df_topic_keywords['topic_theme'] = Topics_theme

In [10]:
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,...,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19,topic_theme
Topic 0,make,work,watch,good,look,talk,know,join,like,health,...,day,address,forward,just,plan,congratulation,new,start,biden,a
Topic 1,right,day,people,let,stand,act,just,read,end,long,...,power,ask,continue,want,stop,gun,congress,violence,attack,b
Topic 2,president,people,say,obama,time,trump,american,family,today,way,...,think,leader,speak,happy,tell,america,community,thing,want,c
Topic 3,year,com,vote,win,election,realdonaldtrump,kamalaharris,brasil,trump,student,...,leave,south,city,remember,govt,poll,voter,china,republican,d
Topic 4,que,thank,para,need,presidente,fight,temer,michel,mais,ncia,...,governo,uma,dont,young,donald,hoje,life,matter,estamos,e
Topic 5,change,live,news,life,don,climate,believe,share,report,bring,...,justice,lose,late,potus,school,medium,work,est,rio,f
Topic 6,india,woman,job,joebiden,nation,tulsigabbard,house,democrat,debate,hear,...,hold,pass,number,chance,like,ready,pradesh,glad,write,g
Topic 7,country,narendramodi,shashitharoor,bjp,arvindkejriwal,best,meet,congress,proud,indian,...,today,delhi,visit,incindia,united,modi,add,break,speech,h
Topic 8,amp,support,big,party,economy,great,meeting,run,important,week,...,history,lead,meetthepress,deal,miss,good,war,candidate,case,i
Topic 9,great,help,today,campaign,pay,thanks,wish,friend,million,child,...,team,strong,high,future,joe,tax,record,serve,know,j


In [11]:
df_topic_keywords.set_index('topic_theme', inplace=True)

In [12]:
df_topic_keywords.T

topic_theme,a,b,c,d,e,f,g,h,i,j
Word 0,make,right,president,year,que,change,india,country,amp,great
Word 1,work,day,people,com,thank,live,woman,narendramodi,support,help
Word 2,watch,people,say,vote,para,news,job,shashitharoor,big,today
Word 3,good,let,obama,win,need,life,joebiden,bjp,party,campaign
Word 4,look,stand,time,election,presidente,don,nation,arvindkejriwal,economy,pay
Word 5,talk,act,trump,realdonaldtrump,fight,climate,tulsigabbard,best,great,thanks
Word 6,know,just,american,kamalaharris,temer,believe,house,meet,meeting,wish
Word 7,join,read,family,brasil,michel,share,democrat,congress,run,friend
Word 8,like,end,today,trump,mais,report,debate,proud,important,million
Word 9,health,long,way,student,ncia,bring,hear,indian,week,child


In [13]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = df_topic_keywords.T.columns
# topicnames = ["Topic" + str(i) for i in range(20)]

# index names
docnames = ["Tweet" + str(i) for i in range(len(df_clean))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

In [14]:
df_document_topic

topic_theme,a,b,c,d,e,f,g,h,i,j,dominant_topic
Tweet0,0.01,0.12,0.11,0.01,0.01,0.60,0.01,0.01,0.11,0.01,5
Tweet1,0.05,0.05,0.05,0.05,0.55,0.05,0.05,0.05,0.05,0.05,4
Tweet2,0.01,0.16,0.01,0.01,0.01,0.01,0.73,0.01,0.01,0.01,6
Tweet3,0.54,0.01,0.01,0.01,0.01,0.13,0.27,0.01,0.01,0.01,0
Tweet4,0.02,0.02,0.68,0.02,0.02,0.02,0.02,0.18,0.02,0.02,2
Tweet5,0.01,0.30,0.01,0.01,0.01,0.30,0.30,0.01,0.01,0.01,1
Tweet6,0.01,0.12,0.01,0.01,0.68,0.01,0.12,0.01,0.01,0.01,4
Tweet7,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.37,0.03,0.37,7
Tweet8,0.38,0.01,0.01,0.01,0.01,0.01,0.17,0.01,0.37,0.01,0
Tweet9,0.01,0.01,0.01,0.01,0.01,0.14,0.01,0.01,0.01,0.76,9


In [15]:
df_document_topic.reset_index(inplace=True)
df_sent_topic= pd.merge(df_clean, df_document_topic, left_index=True, right_index=True)
df_sent_topic.drop('index', axis=1, inplace=True)

In [16]:
df_sent_topic

,tweet_text,a,b,c,d,e,f,g,h,i,j,dominant_topic
0,rt you say what punishment of god be not gift ...,0.01,0.12,0.11,0.01,0.01,0.60,0.01,0.01,0.11,0.01,5
1,carlarhoffmann thank you,0.05,0.05,0.05,0.05,0.55,0.05,0.05,0.05,0.05,0.05,4
2,one of my favorite excerpt from my interview w...,0.01,0.16,0.01,0.01,0.01,0.01,0.73,0.01,0.01,0.01,6
3,come see me in winnipeg on sunday sept for a f...,0.54,0.01,0.01,0.01,0.01,0.13,0.27,0.01,0.01,0.01,0
4,rt megan rapinoe say kneeling during the natio...,0.02,0.02,0.68,0.02,0.02,0.02,0.02,0.18,0.02,0.02,2
5,excite to interview meganrapinoe live tonight ...,0.01,0.30,0.01,0.01,0.01,0.30,0.30,0.01,0.01,0.01,1
6,thank you for all the lovely message about my ...,0.01,0.12,0.01,0.01,0.68,0.01,0.12,0.01,0.01,0.01,4
7,thanks for the cover ew,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.37,0.03,0.37,7
8,a great new book call weareeverywhere just com...,0.38,0.01,0.01,0.01,0.01,0.01,0.17,0.01,0.37,0.01,0
9,rt today mark bradford be one of america ’ s m...,0.01,0.01,0.01,0.01,0.01,0.14,0.01,0.01,0.01,0.76,9


In [17]:
df_topic_theme = df_sent_topic[['tweet_text', 'dominant_topic']]

In [18]:
df_topic_theme.head(10)

,tweet_text,dominant_topic
0,rt you say what punishment of god be not gift ...,5
1,carlarhoffmann thank you,4
2,one of my favorite excerpt from my interview w...,6
3,come see me in winnipeg on sunday sept for a f...,0
4,rt megan rapinoe say kneeling during the natio...,2
5,excite to interview meganrapinoe live tonight ...,1
6,thank you for all the lovely message about my ...,4
7,thanks for the cover ew,7
8,a great new book call weareeverywhere just com...,0
9,rt today mark bradford be one of america ’ s m...,9


In [28]:
i = 0
for tweet in tweets:
    tweet['topic'] = str(df_topic_theme.dominant_topic[i])
    i += 1

In [30]:
with open('English_tweets_with_topic.json', 'w') as f:
    json.dump(tweets, f)